In [146]:
### IMPORTS ###
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE, POST, GET
from pprint import pprint
import csv

In [147]:
### HELPERS ###

def to_valid_iri(iri): 
    return '<' + iri.strip() + '>'

In [148]:
### CONFIGURE CSV WRITER ###
file = open('rdfs_closure_counts.csv', mode='w', newline='')
fieldnames = ['excluded_axiom', 'closure_count', 'reduction_count', 'redundant']
writer = csv.DictWriter(file, fieldnames=fieldnames)
writer.writeheader()
file.flush()

In [149]:
### CONFIGURE SPARQL ENDPOINT REQUESTS ###
class Endpoint:
    query  = SPARQLWrapper('http://localhost:5820/RDFS/query')
    update = SPARQLWrapper('http://localhost:5820/RDFS/update')
    
    def __init__(self):
        pass
endpoint = Endpoint()

In [150]:
### LOAD ALL AXIOMS AND THE FULL CLOSURE ###
endpoint.update.resetQuery()
endpoint.query.clearParameter('reasoning')
endpoint.query.addParameter('reasoning', 'false')
endpoint.query.setReturnFormat(JSON)
endpoint.query.setMethod(GET)
endpoint.query.setQuery("""
    SELECT ?sub ?pred ?obj WHERE {
        ?sub ?pred ?obj .
        FILTER(
            !REGEX(STR(?sub), 'http://www.w3.org/2002/07/owl') &&
            !REGEX(STR(?pred), 'http://www.w3.org/2002/07/owl') &&
            !REGEX(STR(?obj), 'http://www.w3.org/2002/07/owl')
        )
    }
""")
axioms = endpoint.query.query().convert()['results']['bindings']
axioms_count = len(axioms)

endpoint.update.resetQuery()
endpoint.query.clearParameter('reasoning')
endpoint.query.addParameter('reasoning', 'true')
endpoint.query.setReturnFormat(JSON)
endpoint.query.setMethod(GET)
endpoint.query.setQuery("""
    SELECT ?sub ?pred ?obj WHERE {
        ?sub ?pred ?obj .
        FILTER(
            !REGEX(STR(?sub), 'http://www.w3.org/2002/07/owl') &&
            !REGEX(STR(?pred), 'http://www.w3.org/2002/07/owl') &&
            !REGEX(STR(?obj), 'http://www.w3.org/2002/07/owl')
        )
    }
""")
full_closure = endpoint.query.query().convert()['results']['bindings']
full_closure_count = len(full_closure)

In [151]:
axioms_count, full_closure_count

(50, 95)

In [152]:
### CALCULATE CLOSURE WITH ONE EXCLUDED AXIOM ###
for axiom in axioms:
    # REMOVE ONE AXIOM #
    endpoint.update.resetQuery()
    endpoint.update.setMethod(POST)
    delete_query = """
        DELETE WHERE {{
            {sub} {pred} {obj} .
        }}
    """.format(
            sub=to_valid_iri(axiom['sub']['value']),
            pred=to_valid_iri(axiom['pred']['value']), 
            obj=to_valid_iri(axiom['obj']['value'])
    )
    endpoint.update.setQuery(delete_query)
    delete_result = endpoint.update.query()
    
    # GET CLOSURE COUNT #
    endpoint.query.resetQuery()
    endpoint.query.clearParameter('reasoning')
    endpoint.query.addParameter('reasoning', 'true')
    endpoint.query.setReturnFormat(JSON)
    endpoint.update.setMethod(GET)
    endpoint.query.setQuery("""
        SELECT ?sub ?pred ?obj WHERE {
            ?sub ?pred ?obj .
            FILTER(
                !REGEX(STR(?sub), 'http://www.w3.org/2002/07/owl') &&
                !REGEX(STR(?pred), 'http://www.w3.org/2002/07/owl') &&
                !REGEX(STR(?obj), 'http://www.w3.org/2002/07/owl')
            )
        }
    """)
    closure = endpoint.query.query().convert()['results']['bindings']
    closure_count = len(closure)
    
    # GET REDUCTION COUNT #
    endpoint.query.resetQuery()
    endpoint.query.clearParameter('reasoning')
    endpoint.query.addParameter('reasoning', 'false')
    endpoint.query.setReturnFormat(JSON)
    endpoint.update.setMethod(GET)
    endpoint.query.setQuery("""
        SELECT ?sub ?pred ?obj WHERE {
            ?sub ?pred ?obj .
            FILTER(
                !REGEX(STR(?sub), 'http://www.w3.org/2002/07/owl') &&
                !REGEX(STR(?pred), 'http://www.w3.org/2002/07/owl') &&
                !REGEX(STR(?obj), 'http://www.w3.org/2002/07/owl')
            )
        }
    """)
    reduction = endpoint.query.query().convert()['results']['bindings']
    reduction_count = len(reduction)
    
    # FLAG REDUNDANT AXIOMS #
    redudant = closure_count == full_closure_count

    # RECORD TRANSACTION #
    transaction = {
        'excluded_axiom': '{sub} {pred} {obj}'.format(
            sub=to_valid_iri(axiom['sub']['value']),
            pred=to_valid_iri(axiom['pred']['value']), 
            obj=to_valid_iri(axiom['obj']['value'])
        ),
        'closure_count': closure_count,
        'reduction_count': reduction_count,
        'redundant': redudant
    }
    pprint(transaction)
    writer.writerow(transaction)
    
    # ADD BACK AXIOM #
    endpoint.update.resetQuery()
    endpoint.update.setReturnFormat(JSON)
    endpoint.update.setMethod(POST)
    update_query = """
        INSERT DATA {{
            {sub} {pred} {obj}
        }}
    """.format(
            sub=to_valid_iri(axiom['sub']['value']),
            pred=to_valid_iri(axiom['pred']['value']), 
            obj=to_valid_iri(axiom['obj']['value'])
    )
    endpoint.update.setQuery(update_query)
    update_result = endpoint.update.query()

file.close()

{'closure_count': 94,
 'excluded_axiom': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> '
                   '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> '
                   '<http://www.w3.org/1999/02/22-rdf-syntax-ns#Property>',
 'reduction_count': 49,
 'redundant': False}
{'closure_count': 85,
 'excluded_axiom': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> '
                   '<http://www.w3.org/2000/01/rdf-schema#domain> '
                   '<http://www.w3.org/2000/01/rdf-schema#Resource>',
 'reduction_count': 49,
 'redundant': False}
{'closure_count': 94,
 'excluded_axiom': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> '
                   '<http://www.w3.org/2000/01/rdf-schema#range> '
                   '<http://www.w3.org/2000/01/rdf-schema#Class>',
 'reduction_count': 49,
 'redundant': False}
{'closure_count': 94,
 'excluded_axiom': '<http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> '
                   '<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty

{'closure_count': 93,
 'excluded_axiom': '<http://www.w3.org/2000/01/rdf-schema#subClassOf> '
                   '<http://www.w3.org/2000/01/rdf-schema#range> '
                   '<http://www.w3.org/2000/01/rdf-schema#Class>',
 'reduction_count': 49,
 'redundant': False}
{'closure_count': 94,
 'excluded_axiom': '<http://www.w3.org/2000/01/rdf-schema#member> '
                   '<http://www.w3.org/2000/01/rdf-schema#domain> '
                   '<http://www.w3.org/2000/01/rdf-schema#Resource>',
 'reduction_count': 49,
 'redundant': False}
{'closure_count': 94,
 'excluded_axiom': '<http://www.w3.org/2000/01/rdf-schema#member> '
                   '<http://www.w3.org/2000/01/rdf-schema#range> '
     

limit_output extension: Maximum message size of 10000 exceeded with 10932 characters